In [2]:
import pandas as pd
from espn_api.football import League
import math

# Initialize the league
pgb_league = League(league_id=1328496691, year=2024, espn_s2='AEBYECasftI1k7UT5d4A7FbT1EfGRH660hjw%2FclbTqlKTY1a7KlqHIhaj4%2BoPD82tbfS8uOCS0nq%2B9G8wgnOuztJOi%2BCRcAIfue1xsW3vQUYDLJ8TlX9s5ieZPAcI3MvgLzDfzRkT0QWjEbq2MRZrzEnv2zM8jY3MepkKDtXWwE4aauJTL4d4Tq4Zeljxgsz7%2BH4KYkVBSkczEsQPUQldvmdNQmkGWU2E8nQwXgDVAawjYObLsqqO93cuJiu2qlbPsrR7b%2FfEVNn7b9%2FFRMyvmIG8ZUKvUFAe6iINuUrUd0779KsdgCtBbg6210B4W%2FQjgQ%3D', swid='{2C7AFCC4-F961-4FAB-B021-DD728F1DC1E9}')

# Retrieve all free agents
free_agents = pgb_league.free_agents()

# With the value ratio

In [3]:
# Define injury status values
injury_status_values = {
    'HEALTHY': 1.0,
    'DOUBTFUL': 0.75,
    'QUESTIONABLE': 0.5,
    'INJURY_RESERVE': 0.5,
    'OUT': 0.0
}

# Extract relevant stats for each player
free_agents_data = []
for player in free_agents:
    injury_status = tuple(player.injuryStatus) if isinstance(player.injuryStatus, list) else player.injuryStatus
    injury_value = injury_status_values.get(injury_status, 1.0)

    player_data = {
        'Name': player.name,
        'Points': player.points,
        'Projected': player.projected_points,
        'Position': player.position,
        'Team': player.proTeam,
        'Injury Status': injury_status,
        'Eligible Slots': player.eligibleSlots,
        'Acquisition Type': player.acquisitionType,
        'Injury Value': injury_value
    }
    free_agents_data.append(player_data)

# Create a pandas DataFrame from the extracted data
free_agents_df = pd.DataFrame(free_agents_data)

# Calculate the Value Ratio
free_agents_df['Value Ratio'] = free_agents_df['Projected'] / free_agents_df['Points']

# Add a new column for the ratio of eligible positions to projected points, factoring in injury status and value ratio
free_agents_df['PPR'] = (free_agents_df['Eligible Slots'].apply(lambda x: len(x)) / free_agents_df['Projected']) * free_agents_df['Injury Value'] * free_agents_df['Value Ratio']

# Sort the DataFrame by the new ratio column in descending order
sorted_free_agents_df = free_agents_df.sort_values(by='PPR', ascending=False)

# Display the sorted DataFrame
print(sorted_free_agents_df)

                  Name  Points  Projected Position Team   Injury Status  \
0        Justin Tucker       0       7.83        K  BAL          ACTIVE   
1           49ers D/ST       0       7.02     D/ST   SF              ()   
28         Sam Darnold       0      16.45       QB  MIN    QUESTIONABLE   
29    Chase McLaughlin       0       8.12        K   TB          ACTIVE   
30       Juwan Johnson       0       4.76       TE   NO          ACTIVE   
31      Dustin Hopkins       0       7.58        K  CLE          ACTIVE   
32            Matt Gay       0       7.64        K  IND          ACTIVE   
33          Tyler Bass       0       8.00        K  BUF          ACTIVE   
34     Brenton Strange       0       1.05       TE  JAX          ACTIVE   
36      Michael Wilson       0       8.39       WR  ARI          ACTIVE   
37      Khalil Herbert       0       2.08       RB  CHI          ACTIVE   
38    Dontayvion Wicks       0       6.66       WR   GB          ACTIVE   
39         Eagles D/ST   

In [4]:
# Save the DataFrame to a CSV file
sorted_free_agents_df.to_csv('free_agents_2.csv', index=False)

# Without the value ratio

In [5]:
# Define injury status values
injury_status_values = {
    'HEALTHY': 1.0,
    'DOUBTFUL': 0.75,
    'QUESTIONABLE': 0.5,
    'INJURY_RESERVE': 0.5,
    'OUT': 0.0
}

# Extract relevant stats for each player
free_agents_data = []
for player in free_agents:
    injury_status = tuple(player.injuryStatus) if isinstance(player.injuryStatus, list) else player.injuryStatus
    injury_value = injury_status_values.get(injury_status, 1.0)

    player_data = {
        'Name': player.name,
        'Points': player.points,
        'Projected': player.projected_points,
        'Position': player.position,
        'Team': player.proTeam,
        'Injury Status': injury_status,
        'Eligible Slots': player.eligibleSlots,
        'Acquisition Type': player.acquisitionType,
        'Injury Value': injury_value
    }
    free_agents_data.append(player_data)

# Create a pandas DataFrame from the extracted data
free_agents_df = pd.DataFrame(free_agents_data)

# Calculate the number of eligible slots for each player
free_agents_eligible_slots = free_agents_df['Eligible Slots'].apply(lambda x: len(x))

# Convert free_agents_eligible_slots to a float
free_agents_eligible_slots_float = free_agents_eligible_slots.astype(float)

# Add a new column for the ratio of eligible positions to projected points, factoring in injury status
free_agents_df['PPR'] = ((free_agents_eligible_slots_float + free_agents_df['Injury Value']) + ((free_agents_df['Projected'] - free_agents_df['Points']) / free_agents_df['Points']))

# Sort the DataFrame by the new ratio column in descending order
sorted_free_agents_df = free_agents_df.sort_values(by='PPR', ascending=False)

# Display the sorted DataFrame
print(sorted_free_agents_df)

                  Name  Points  Projected Position Team   Injury Status  \
0        Justin Tucker       0       7.83        K  BAL          ACTIVE   
1           49ers D/ST       0       7.02     D/ST   SF              ()   
28         Sam Darnold       0      16.45       QB  MIN    QUESTIONABLE   
29    Chase McLaughlin       0       8.12        K   TB          ACTIVE   
30       Juwan Johnson       0       4.76       TE   NO          ACTIVE   
31      Dustin Hopkins       0       7.58        K  CLE          ACTIVE   
32            Matt Gay       0       7.64        K  IND          ACTIVE   
33          Tyler Bass       0       8.00        K  BUF          ACTIVE   
34     Brenton Strange       0       1.05       TE  JAX          ACTIVE   
36      Michael Wilson       0       8.39       WR  ARI          ACTIVE   
37      Khalil Herbert       0       2.08       RB  CHI          ACTIVE   
38    Dontayvion Wicks       0       6.66       WR   GB          ACTIVE   
39         Eagles D/ST   

In [6]:
# Save the DataFrame to a CSV file
sorted_free_agents_df.to_csv('free_agents4.csv', index=False)

In [7]:
import pandas as pd
from espn_api.football import League

def extract_free_agents_data(free_agents, injury_status_values):
    free_agents_data = []
    for player in free_agents:
        injury_status = player.injuryStatus if not isinstance(player.injuryStatus, list) else tuple(player.injuryStatus)
        injury_value = injury_status_values.get(injury_status, 1.0)

        player_data = {
            'Name': player.name,
            'Points': player.points,
            'Projected': player.projected_points,
            'Position': player.position,
            'Team': player.proTeam,
            'Injury Status': injury_status,
            'Eligible Slots': player.eligibleSlots,
            'Acquisition Type': player.acquisitionType,
            'Injury Value': injury_value
        }
        free_agents_data.append(player_data)
    return pd.DataFrame(free_agents_data)

def calculate_ppr_with_value_ratio(df):
    df['Value Ratio'] = df['Projected'] / df['Points']
    df['PPR'] = (df['Eligible Slots'].apply(lambda x: len(x)) / df['Projected']) * df['Injury Value'] * df['Value Ratio']
    return df.sort_values(by='PPR', ascending=False)

def calculate_ppr_without_value_ratio(df):
    eligible_slots_float = df['Eligible Slots'].apply(lambda x: len(x)).astype(float)
    df['PPR'] = ((eligible_slots_float + df['Injury Value']) + ((df['Projected'] - df['Points']) / df['Points']))
    return df.sort_values(by='PPR', ascending=False)

# Initialize the league
pgb_league = League(league_id=1328496691, year=2024, espn_s2='AEBYECasftI1k7UT5d4A7FbT1EfGRH660hjw%2FclbTqlKTY1a7KlqHIhaj4%2BoPD82tbfS8uOCS0nq%2B9G8wgnOuztJOi%2BCRcAIfue1xsW3vQUYDLJ8TlX9s5ieZPAcI3MvgLzDfzRkT0QWjEbq2MRZrzEnv2zM8jY3MepkKDtXWwE4aauJTL4d4Tq4Zeljxgsz7%2BH4KYkVBSkczEsQPUQldvmdNQmkGWU2E8nQwXgDVAawjYObLsqqO93cuJiu2qlbPsrR7b%2FfEVNn7b9%2FFRMyvmIG8ZUKvUFAe6iINuUrUd0779KsdgCtBbg6210B4W%2FQjgQ%3D', swid='{2C7AFCC4-F961-4FAB-B021-DD728F1DC1E9}')

# Retrieve all free agents
free_agents = pgb_league.free_agents()

# Define injury status values
injury_status_values = {
    'HEALTHY': 1.0,
    'DOUBTFUL': 0.75,
    'QUESTIONABLE': 0.5,
    'INJURY_RESERVE': 0.5,
    'OUT': 0.0
}

# Extract data and create DataFrame
free_agents_df = extract_free_agents_data(free_agents, injury_status_values)

# Calculate PPR with value ratio
sorted_free_agents_df_with_value_ratio = calculate_ppr_with_value_ratio(free_agents_df)
print(sorted_free_agents_df_with_value_ratio)
sorted_free_agents_df_with_value_ratio.to_csv('free_agents_2.csv', index=False)

# Calculate PPR without value ratio
sorted_free_agents_df_without_value_ratio = calculate_ppr_without_value_ratio(free_agents_df)
print(sorted_free_agents_df_without_value_ratio)
sorted_free_agents_df_without_value_ratio.to_csv('free_agents4.csv', index=False)

                  Name  Points  Projected Position Team   Injury Status  \
0        Justin Tucker       0       7.83        K  BAL          ACTIVE   
1           49ers D/ST       0       7.02     D/ST   SF              ()   
28         Sam Darnold       0      16.45       QB  MIN    QUESTIONABLE   
29    Chase McLaughlin       0       8.12        K   TB          ACTIVE   
30       Juwan Johnson       0       4.76       TE   NO          ACTIVE   
31      Dustin Hopkins       0       7.58        K  CLE          ACTIVE   
32            Matt Gay       0       7.64        K  IND          ACTIVE   
33          Tyler Bass       0       8.00        K  BUF          ACTIVE   
34     Brenton Strange       0       1.05       TE  JAX          ACTIVE   
36      Michael Wilson       0       8.39       WR  ARI          ACTIVE   
37      Khalil Herbert       0       2.08       RB  CHI          ACTIVE   
38    Dontayvion Wicks       0       6.66       WR   GB          ACTIVE   
39         Eagles D/ST   

In [ ]:
# Extract data and create DataFrame
free_agents_df = extract_free_agents_data(free_agents, injury_status_values)

# Calculate PPR with value ratio
sorted_free_agents_df_with_value_ratio = calculate_ppr_with_value_ratio(free_agents_df)
print(sorted_free_agents_df_with_value_ratio)
sorted_free_agents_df_with_value_ratio.to_csv('free_agents_2.csv', index=False)

# Calculate PPR without value ratio
sorted_free_agents_df_without_value_ratio = calculate_ppr_without_value_ratio(free_agents_df)
print(sorted_free_agents_df_without_value_ratio)
sorted_free_agents_df_without_value_ratio.to_csv('free_agents4.csv', index=False)